In [10]:
import pandas as pd

df = pd.read_csv("Data-GP1_with_features.csv")

1) Define the objective + guard against leakage

Decide what you can know at prediction time. If you’re quoting a price at purchase time, drop any features that peek into the future (e.g., COVID around departure if not known yet).

In [ ]:
import numpy as np
import pandas as pd

# Target (log helps linear models + stabilizes variance)
TARGET = "mean_net_ticket_price"
df = df.copy()
df["y_log"] = np.log1p(df[TARGET])

num_feats = [
    "lead_time_days",
    "Culmulative_sales","route_sales_rank","sales_diff_1",
    "price_to_route_month_mean","route_month_mean_price",
    "covid_cases_at_purchase","stringency_at_purchase",
]

bool_feats = [
    "isReturn","isOneway","isNormCabin",
    "is_weekend_dept","is_weekend_purchase",
    "is_holiday_dept","is_holiday_purchase",
    "is_sg_holiday_dept","is_us_holiday_dept","is_cn_holiday_dept","is_fr_holiday_dept","is_in_holiday_dept","is_ru_holiday_dept",
    "is_sg_holiday_purchase","is_us_holiday_purchase","is_cn_holiday_purchase","is_fr_holiday_purchase","is_in_holiday_purchase","is_ru_holiday_purchase",
    "return_x_norm","oneway_x_norm","return_x_weekend","oneway_x_weekend",
]

cat_feats = [
    "dept_weekday","purchase_weekday","dept_season",
    "Train_Number_All","Customer_Cat"
]

# Keep only columns that exist
num_feats  = [c for c in num_feats  if c in df.columns]
bool_feats = [c for c in bool_feats if c in df.columns]
cat_feats  = [c for c in cat_feats  if c in df.columns]

X_cols = num_feats + bool_feats + cat_feats
print("Using features:", X_cols)


Using features: ['lead_time_days', 'Culmulative_sales', 'route_sales_rank', 'sales_diff_1', 'price_to_route_month_mean', 'route_month_mean_price', 'isReturn', 'isOneway', 'isNormCabin', 'is_weekend_dept', 'is_weekend_purchase', 'is_holiday_dept', 'is_holiday_purchase', 'return_x_norm', 'oneway_x_norm', 'return_x_weekend', 'oneway_x_weekend', 'dept_weekday', 'purchase_weekday', 'dept_season', 'Train_Number_All', 'Customer_Cat']


Time-aware train/test split

Don’t shuffle randomly when there’s time involved.

In [12]:
# Ensure dates are datetime
for c in ["Purchase_Date","Dept_Date"]:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")

df_sorted = df.sort_values("Purchase_Date").reset_index(drop=True)

cut = int(len(df_sorted) * 0.8)  # 80% train, 20% test (time ordered)
train = df_sorted.iloc[:cut].copy()
test  = df_sorted.iloc[cut:].copy()

X_train, y_train = train[X_cols], train["y_log"]
X_test,  y_test  = test[X_cols],  test["y_log"]

print(train["Purchase_Date"].min(), "→", train["Purchase_Date"].max(), "| TRAIN")
print(test["Purchase_Date"].min(),  "→", test["Purchase_Date"].max(),  "| TEST")


2018-06-01 00:00:00 → 2019-04-22 00:00:00 | TRAIN
2019-04-22 00:00:00 → 2019-06-30 00:00:00 | TEST


Preprocess + LASSO (with CV)

Scale numeric/boolean, one-hot categoricals, and run LassoCV.

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LassoCV, ElasticNetCV, LinearRegression
from sklearn.pipeline import Pipeline
import numpy as np

numeric = num_feats + bool_feats   # scale both numeric + 0/1 flags
categorical = cat_feats

pre = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=True, with_std=True), numeric),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical),
    ],
    remainder="drop"
)

alphas = np.logspace(-4, 1.5, 80)  # search from 1e-4 to ~31
lasso = LassoCV(alphas=alphas, cv=5, random_state=42, n_jobs=-1, max_iter=20000)

pipe_lasso = Pipeline([
    ("pre", pre),
    ("model", lasso)
])

pipe_lasso.fit(X_train, y_train)

print("Best alpha (LASSO):", pipe_lasso.named_steps["model"].alpha_)
print("Non-zero coefs:", np.sum(pipe_lasso.named_steps["model"].coef_ != 0))


Best alpha (LASSO): 0.0008036432311789218
Non-zero coefs: 28


Evaluate (log space + back-transform)

Report MAE/RMSE/ MAP E on the original price scale.

In [17]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluate(pipe, X_tr, y_tr, X_te, y_te, label="Model"):
    # predictions in log1p space
    pred_tr = pipe.predict(X_tr)
    pred_te = pipe.predict(X_te)

    # true values (original scale)
    y_tr_true = np.expm1(y_tr)
    y_te_true = np.expm1(y_te)

    # naive back-transform
    y_tr_hat_naive = np.expm1(pred_tr)
    y_te_hat_naive = np.expm1(pred_te)

    # Duan smearing correction for log1p models
    resid_tr = y_tr - pred_tr                       # residuals in log space
    smear = float(np.mean(np.exp(resid_tr)))        # E[exp(e)]
    y_tr_hat_smear = smear * np.exp(pred_tr) - 1.0
    y_te_hat_smear = smear * np.exp(pred_te) - 1.0

    # clip tiny negatives
    y_tr_hat_naive = np.clip(y_tr_hat_naive, 0, None)
    y_te_hat_naive = np.clip(y_te_hat_naive, 0, None)
    y_tr_hat_smear = np.clip(y_tr_hat_smear, 0, None)
    y_te_hat_smear = np.clip(y_te_hat_smear, 0, None)

    def rmse(y_true, y_pred):
        return float(np.sqrt(mean_squared_error(y_true, y_pred)))

    def mape(y_true, y_pred):
        return float(np.mean(np.abs((y_true - y_pred) / np.clip(y_true, 1e-8, None))) * 100)

    metrics = {
        # log-space fit (your original intent)
        "R2_tr_log": r2_score(y_tr, pred_tr),
        "R2_te_log": r2_score(y_te, pred_te),

        # original-scale (naive back-transform)
        "MAE_tr": mean_absolute_error(y_tr_true, y_tr_hat_naive),
        "RMSE_tr": rmse(y_tr_true, y_tr_hat_naive),
        "MAPE_tr_%": mape(y_tr_true, y_tr_hat_naive),
        "R2_tr_orig": r2_score(y_tr_true, y_tr_hat_naive),

        "MAE_te": mean_absolute_error(y_te_true, y_te_hat_naive),
        "RMSE_te": rmse(y_te_true, y_te_hat_naive),
        "MAPE_te_%": mape(y_te_true, y_te_hat_naive),
        "R2_te_orig": r2_score(y_te_true, y_te_hat_naive),

        # smearing-corrected (often better calibrated)
        "MAE_te_smear": mean_absolute_error(y_te_true, y_te_hat_smear),
        "RMSE_te_smear": rmse(y_te_true, y_te_hat_smear),
        "MAPE_te_smear_%": mape(y_te_true, y_te_hat_smear),
        "R2_te_orig_smear": r2_score(y_te_true, y_te_hat_smear),

        "SmearingFactor": smear,
    }

    print(label, metrics)
    return metrics

# usage
m_lasso = evaluate(pipe_lasso, X_train, y_train, X_test, y_test, "LASSO")


LASSO {'R2_tr_log': 0.9433488946681736, 'R2_te_log': 0.9355304007781823, 'MAE_tr': 93187001.67857896, 'RMSE_tr': 38167660650.91753, 'MAPE_tr_%': 1186234.5385857795, 'R2_tr_orig': -6.555408697723465e+16, 'MAE_te': 43.03533610509039, 'RMSE_te': 2591.346246336126, 'MAPE_te_%': 11.383290697231866, 'R2_te_orig': -361.46933405071564, 'MAE_te_smear': 43.234643255332195, 'RMSE_te_smear': 2613.996820373126, 'MAPE_te_smear_%': 11.446320698789643, 'R2_te_orig_smear': -367.8336092234766, 'SmearingFactor': 1.0087088013709464}


Which features did LASSO keep?

Get feature names after preprocessing and list non-zero coefficients (standardized).

In [18]:
# Get transformed feature names
cat_names = pipe_lasso.named_steps["pre"].named_transformers_["cat"].get_feature_names_out(categorical) if categorical else np.array([])
feat_names = np.concatenate([numeric, cat_names])

coefs = pipe_lasso.named_steps["model"].coef_
mask = coefs != 0
selected = pd.DataFrame({
    "feature": feat_names[mask],
    "coef_std": coefs[mask]
}).sort_values("coef_std", key=lambda s: np.abs(s), ascending=False)

print(f"LASSO kept {mask.sum()} / {len(feat_names)} features")
selected.head(30)


LASSO kept 28 / 52 features


,feature,coef_std
4,price_to_route_month_mean,0.459848
5,route_month_mean_price,0.207496
21,Train_Number_All_C,0.044209
0,lead_time_days,-0.043307
8,isNormCabin,-0.040761
18,dept_season_Winter,0.032968
27,Customer_Cat_A,0.027920
1,Culmulative_sales,-0.023946
7,isOneway,-0.016651
2,route_sales_rank,-0.015063


Human-readable formula

Coefficients above are for standardized inputs. To get a simple linear formula in original units, refit a plain LinearRegression on only the selected columns using the same one-hot, but without scaling numeric features.

In [20]:
# Build a 'no-scale' preprocessor (one-hot only), then restrict to selected columns
pre_noscale = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric + bool_feats),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical),
    ],
    remainder="drop"
)

# Fit on TRAIN to get transformed matrix & names
Xtr_n = pre_noscale.fit_transform(X_train)
cat_names_n = pre_noscale.named_transformers_["cat"].get_feature_names_out(categorical) if categorical else np.array([])
feat_names_n = np.concatenate([numeric + bool_feats, cat_names_n])

# Keep only the features LASSO selected (intersection by name)
keep_set = set(selected["feature"])
keep_idx = [i for i, n in enumerate(feat_names_n) if n in keep_set]

Xtr_sel = Xtr_n[:, keep_idx]
Xte_sel = pre_noscale.transform(X_test)[:, keep_idx]
feat_sel_names = [feat_names_n[i] for i in keep_idx]

ols = LinearRegression()
ols.fit(Xtr_sel, y_train)

coef = pd.DataFrame({"feature": feat_sel_names, "coef": ols.coef_}).sort_values("coef", key=lambda s: np.abs(s), ascending=False)
intercept = ols.intercept_

print("Linear model (log-price) with selected features")
print("Intercept:", intercept)
coef.head(30)


Linear model (log-price) with selected features
Intercept: 3.7438492233079774


,feature,coef
4,price_to_route_month_mean,0.796879
29,Train_Number_All_C,0.052980
8,isNormCabin,-0.041483
16,isNormCabin,-0.041483
26,dept_season_Winter,0.037961
34,Train_Number_All_K,-0.037427
20,oneway_x_norm,0.031556
12,oneway_x_norm,0.031556
35,Customer_Cat_A,0.031231
15,isOneway,-0.029947


Your formula in log space is:
log(1 + price) = Intercept + Σ coef_i * feature_i
To predict price: price_hat = exp(Intercept + Σ coef_i * feature_i) - 1.

(If you want statistical p-values / confidence intervals, swap LinearRegression with statsmodels.api.OLS on Xtr_sel and y_train.)

Which ones to keep? (beyond LASSO)

Permutation importance (model-agnostic): keeps features that truly affect predictions on the holdout.

Stability selection (bootstrap): keeps features repeatedly chosen across resamples.

In [21]:
from sklearn.inspection import permutation_importance

perm = permutation_importance(pipe_lasso, X_test, y_test, n_repeats=40, random_state=42, n_jobs=-1)
perm_imp = pd.DataFrame({
    "feature": feat_names,
    "import_mean": perm.importances_mean,
    "import_std": perm.importances_std
}).sort_values("import_mean", ascending=False)

perm_imp.head(20)


ValueError: All arrays must be of the same length